In [2]:
import datetime
import pandas as pd
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import GOOG
import talib
import yfinance as yf

In [34]:
aapl = yf.Ticker("aapl")
aapl_historical = aapl.history(start="2020-06-02", end="2023-06-07")
aapl_historical

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-06-02 00:00:00-04:00,78.312329,78.969104,77.867968,78.944687,87642800,0.0,0.0
2020-06-03 00:00:00-04:00,79.266971,79.642970,78.690765,79.379280,104491200,0.0,0.0
2020-06-04 00:00:00-04:00,79.201048,79.501353,78.319649,78.695648,87560400,0.0,0.0
2020-06-05 00:00:00-04:00,78.947114,80.998004,78.917817,80.936966,137250400,0.0,0.0
2020-06-08 00:00:00-04:00,80.631803,81.449720,79.916434,81.415535,95654400,0.0,0.0
...,...,...,...,...,...,...,...
2023-05-31 00:00:00-04:00,176.393208,178.402541,175.826212,176.313629,99625300,0.0,0.0
2023-06-01 00:00:00-04:00,176.761252,179.168466,175.995316,179.138626,68901800,0.0,0.0
2023-06-02 00:00:00-04:00,180.073659,180.819697,178.313005,179.994080,61945900,0.0,0.0


In [35]:
def MeanReversionStrat(data, range):
    class MeanReversion(Strategy):

        # Do as much initial computation as possible
        def init(self):
            self.sma = self.I(talib.SMA, self.data.Close, range)


        # Step through bars one by one
        # Note that multiple buys are a thing here

        def next(self):
            if crossover(self.data.Close, self.sma):
                self.position.close()
            elif crossover(self.sma, self.data.Close):
                self.buy()


    bt = Backtest(data, MeanReversion, cash=10_000)
    stats =  bt.run()
    print(stats)

In [36]:
def BollingerStart(data, range, upperbound, lowerbound):
    class BollingerReversion(Strategy):
        # Do as much initial computation as possible
        def init(self):
            self.bands = self.I(talib.BBANDS, self.data.Close, range, upperbound, lowerbound)

        # Step through bars one by one
        # Note that multiple buys are a thing here

        def next(self):
            if crossover(self.data.Close, self.bands[2][-1]):
                self.position.close()
            elif crossover(self.bands[0][-1], self.data.Close):
                self.buy()


    bt = Backtest(data, BollingerReversion, cash=10_000)
    stats =  bt.run()
    print(stats)

In [37]:
#MeanReversionStrat(aapl_historical,20)

Start                     2020-06-02 00:00...
End                       2023-06-06 00:00...
Duration                   1099 days 00:00:00
Exposure Time [%]                   43.346509
Equity Final [$]                 10798.204512
Equity Peak [$]                  12374.559272
Return [%]                           7.982045
Buy & Hold Return [%]              125.807838
Return (Ann.) [%]                    2.582491
Volatility (Ann.) [%]               22.267679
Sharpe Ratio                         0.115975
Sortino Ratio                        0.173935
Calmar Ratio                         0.096829
Max. Drawdown [%]                  -26.670519
Avg. Drawdown [%]                   -7.614597
Max. Drawdown Duration      844 days 00:00:00
Avg. Drawdown Duration      162 days 00:00:00
# Trades                                   36
Win Rate [%]                        72.222222
Best Trade [%]                       3.898095
Worst Trade [%]                     -8.662729
Avg. Trade [%]                    

In [38]:
#BollingerStart(aapl_historical,20,1,1)

Start                     2020-06-02 00:00...
End                       2023-06-06 00:00...
Duration                   1099 days 00:00:00
Exposure Time [%]                   49.670619
Equity Final [$]                 18852.652062
Equity Peak [$]                  18955.007062
Return [%]                          88.526521
Buy & Hold Return [%]              125.807838
Return (Ann.) [%]                   23.432069
Volatility (Ann.) [%]               26.030854
Sharpe Ratio                         0.900165
Sortino Ratio                        1.774115
Calmar Ratio                         1.395369
Max. Drawdown [%]                  -16.792742
Avg. Drawdown [%]                   -4.727519
Max. Drawdown Duration      288 days 00:00:00
Avg. Drawdown Duration       51 days 00:00:00
# Trades                                   12
Win Rate [%]                        58.333333
Best Trade [%]                      32.520406
Worst Trade [%]                      -7.29221
Avg. Trade [%]                    